#Team United:

Samer Baslan <br>
Rohan Kumar <br>
Jimmy Liang <br>
Tripura Chandana Gayatri Gorla

##Installs

In [1]:
!pip install finpie -qqq
!pip install newsapi-python -qqq

     |████████████████████████████████| 40.1 MB 1.2 MB/s 
     |████████████████████████████████| 958 kB 42.6 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 356 kB 51.6 MB/s 
     |████████████████████████████████| 138 kB 43.7 MB/s 
     |████████████████████████████████| 3.6 MB 42.8 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 1.1 MB 37.7 MB/s 
     |████████████████████████████████| 160 kB 57.4 MB/s 
     |████████████████████████████████| 192 kB 49.0 MB/s 
     |████████████████████████████████| 271 kB 59.0 MB/s 
     |████████████████████████████████| 802 kB 48.5 MB/s 
     |████████████████████████████████| 132 kB 54.7 MB/s 
     |████████████████████████████████| 793 kB 44.6 MB/s 
     |████████████████████████████████| 793 kB 38.2 MB/s 
     |████████████████████████████████| 791 kB 50.8 MB/s 
     |██████████████

##Imports

In [2]:
import requests
import json
import csv

import finpie as fp
from finpie import NewsData

import newsapi
from newsapi import NewsApiClient

import pandas as pd

/usr/local/lib/python3.7/dist-packages/trio/_core/_multierror.py:464: RuntimeWarning: IPython detected, but you already have a custom exception handler installed. I'll skip installing Trio's custom handler, but this means MultiErrors will not show full tracebacks.
  category=RuntimeWarning,


##Scrape prices

In [64]:
tickerlist = ["AAPL", "NVDA", "TSLA", "AMZN", "GOOGL", "FB", "MSFT", "NFLX"] #add however many we want

In [21]:
def scrapeHistoricalPrices(tickerlist):
  d = {ticker: pd.DataFrame(fp.historical_prices(ticker)) for ticker in tickerlist}
  return d

In [67]:
prices = scrapeHistoricalPrices(tickerlist)
prices

{'AAPL':                   open        high  ...   adj_close     volume
 date                                ...                       
 1980-12-12    0.128348    0.128906  ...    0.100453  469033600
 1980-12-15    0.122210    0.122210  ...    0.095213  175884800
 1980-12-16    0.113281    0.113281  ...    0.088224  105728000
 1980-12-17    0.115513    0.116071  ...    0.090408   86441600
 1980-12-18    0.118862    0.119420  ...    0.093029   73449600
 ...                ...         ...  ...         ...        ...
 2021-11-16  149.940002  151.490005  ...  151.000000   59256200
 2021-11-17  151.000000  155.000000  ...  153.490005   88807000
 2021-11-18  153.710007  158.669998  ...  157.869995  137827700
 2021-11-19  157.649994  161.020004  ...  160.550003  117147500
 2021-11-22  161.679993  165.699997  ...  161.020004  117342000
 
 [10325 rows x 6 columns],
 'AMZN':                    open         high  ...    adj_close    volume
 date                                  ...               

Create CSV files

In [68]:
for name in prices:
  prices[name].to_csv(f"prices_{name}")

In [69]:
test_prices = pd.read_csv("prices_AAPL")
test_prices

,date,open,high,low,close,adj_close,volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100453,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095213,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088224,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090408,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093029,73449600
...,...,...,...,...,...,...,...
10320,2021-11-16,149.940002,151.490005,149.339996,151.000000,151.000000,59256200
10321,2021-11-17,151.000000,155.000000,150.990005,153.490005,153.490005,88807000
10322,2021-11-18,153.710007,158.669998,153.050003,157.869995,157.869995,137827700
10323,2021-11-19,157.649994,161.020004,156.529999,160.550003,160.550003,117147500


##Scrape news articles

In [70]:
newsapi = NewsApiClient(api_key='d2fc944a8f1c46f6a2a16c76ca6a98e5')

In [71]:
tickernames = ["apple", "amazon", "google", "facebook", "tesla", "nvidia", "microsoft", "netflix"]

In [72]:
def scrapeNewsData(tickerlist):
  #d = {ticker: pd.DataFrame(fp.historical_prices(ticker)) for ticker in tickerlist}
  d = {}
  for ticker in tickerlist:
    d[ticker] = newsapi.get_everything(q=ticker,
                                       sources='bloomberg,business-insider',
                                       domains='techcrunch.com',
                                       from_param='2021-10-23', #update from_param since we're using free tier
                                       to='2021-11-15',
                                       language='en',
                                       sort_by='relevancy',
                                       page=2
                                       )
  return d

news = scrapeNewsData(tickernames)

In [73]:
def createNewsDataFrame(dictionary):
  d = {}
  title_list = []
  article_list = []
  source_list = []
  for company in dictionary:
    company_name = company
    for i in range(len(dictionary[company]['articles'])):
      title_list.append(dictionary[company]['articles'][i]['title'])
      article_list.append(dictionary[company]['articles'][i]['content'])
      source_list.append(dictionary[company]['articles'][i]['source']['name'])
      dataframe = pd.DataFrame(data = {'company_name': company_name,
                                      'title': title_list,
                                      'article': article_list,
                                      'source': source_list})
    d[company] = dataframe
    title_list.clear()
    article_list.clear()
    source_list.clear()
  return d

news_dataframes = createNewsDataFrame(news)

In [74]:
news_dataframes

{'amazon':    company_name  ...            source
 0        amazon  ...  Business Insider
 1        amazon  ...  Business Insider
 2        amazon  ...  Business Insider
 3        amazon  ...  Business Insider
 4        amazon  ...  Business Insider
 5        amazon  ...  Business Insider
 6        amazon  ...  Business Insider
 7        amazon  ...  Business Insider
 8        amazon  ...  Business Insider
 9        amazon  ...  Business Insider
 10       amazon  ...  Business Insider
 11       amazon  ...  Business Insider
 12       amazon  ...  Business Insider
 13       amazon  ...  Business Insider
 14       amazon  ...  Business Insider
 15       amazon  ...  Business Insider
 16       amazon  ...  Business Insider
 17       amazon  ...  Business Insider
 18       amazon  ...  Business Insider
 19       amazon  ...  Business Insider
 
 [20 rows x 4 columns], 'apple':    company_name  ...            source
 0         apple  ...  Business Insider
 1         apple  ...  Business Insi

In [62]:
news_dataframes["tesla"]

,company_name,title,article,source
0,tesla,Elon Musk 'didn't even tell his team' he was m...,Tesla's California leadership team didn't find...,Business Insider
1,tesla,Tesla's Model S is no longer the longest-range...,Plenty of startups are working on exciting new...,Business Insider
2,tesla,'The Big Short' investor Michael Burry says El...,Michael Burry.\r\nJim Spellman/Getty Images\r\...,Business Insider
3,tesla,Lucid jumps 9% as deliveries of its Air sedan ...,Lucid Motors\r\n<ul><li>Lucid Group surged as ...,Business Insider
4,tesla,Elon Musk changes his Twitter name to 'Lorde E...,Tesla CEO Elon Musk.\r\nBritta Pedersen/Pool/A...,Business Insider
5,tesla,A Tesla bull who runs an ETF loaded with the E...,"Tad Park, founder and CEO of investment firm V...",Business Insider
6,tesla,A Dutch lab decrypted Tesla's heavily guarded ...,A lab operated by the Dutch government said it...,Business Insider
7,tesla,"Lucid Motors' first car, a $169,000 electric s...","The Air sedan, the first model from electric-c...",Business Insider
8,tesla,"Elon Musk just sold $5 billion in Tesla stock,...",Yichuan Cao/NurPhoto via Getty Images\r\nTesla...,Business Insider
9,tesla,Global shares gain as trillion-dollar Tesla br...,US politicians often place major bets on Wall ...,Business Insider


Create CSV files

In [15]:
for name in news_dataframes:
  news_dataframes[name].to_csv(f"news_{name}", index = False)

In [16]:
test_news = pd.read_csv("news_apple")
test_news

,company_name,title,article,source
0,apple,"Square, seeking life-long users, opens Cash Ap...",The news: Square's \r\nCash App is now letting...,Business Insider
1,apple,"Jared Leto, who plays disgraced WeWork cofound...","The man who plays Joker in the ""Suicide Squad""...",Business Insider
2,apple,New York City Mayor-Elect Eric Adams says he w...,New York City Mayor-Elect Eric Adams took to T...,Business Insider
3,apple,Zia Credit Union review: Hispanic American-led...,Personal Finance Insider writes about products...,Business Insider
4,apple,10 things in tech you need to know,"Hello, world. We've got news about a new way t...",Business Insider
5,apple,5 ways I'm using credit card points to cover 7...,This post contains links to products from our ...,Business Insider
6,apple,Sen. Mitt Romney dresses up as Ted Lasso in Ha...,Sen. Mitt Romney of Utah donned one of this ye...,Business Insider
7,apple,"Apple's Tim Cook, WeWork's Adam Neumann, and C...",Adam Neumann.\r\nMichael Kovac/Getty Images fo...,Business Insider
8,apple,10 things in tech you need to know today,Good morning and welcome to 10 Things in Tech....,Business Insider
9,apple,These are the top 10 stocks being talked about...,Photo illustration by Jakub Porzycki/NurPhoto ...,Business Insider
